In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
def generate_pairs(
    df,
    n_neighbors = 5,
    train = False
):
    '''
    add some documentation here
    '''
    
    orig_df = df.copy()
    
    # define and scale the location df
    location_data = df[['latitude', 'longitude']].values
    location_data = StandardScaler().fit_transform(location_data) 
    
    # define and fit the knn
    knn = NearestNeighbors(n_neighbors = min(n_neighbors + 1, len(df)), 
                           algorithm = 'kd_tree',
                           n_jobs=-1)
    knn.fit(location_data)
    
    # generate the neighbours array - index of closest n_neighbors + 1 samples to each of the samples in the location data array, including that sample istelf
    neighbors_array = knn.kneighbors(location_data,
                                     return_distance=False)

    # create column dictionaries
    cols_1 = dict(zip(orig_df.columns, [f'{col}_1' for col in orig_df.columns]))
    cols_2 = dict(zip(orig_df.columns, [f'{col}_2' for col in orig_df.columns]))

    # create dataframes needed for final join
    df_1 = orig_df
    idxs = neighbors_array.flatten()
    df_2 = df_1.iloc[idxs].reset_index().rename(columns=cols_2)

    # change df_1
    df_1['orig_index'] = df_1.index
    df_1 = pd.concat(knn.n_neighbors * [df_1],
                     ignore_index=True)
    df_1 = df_1.reset_index().rename(columns = cols_1)
    df_1 = df_1.sort_values(['orig_index', 'index'])
    df_1.drop(columns=['orig_index', 'index'],
              inplace=True)
    df_1.reset_index(drop=True, inplace=True)
    
    # create final df
    df = pd.concat([df_1, df_2],
                   axis=1)
    df = df.loc[df.id_1 != df.id_2].reset_index(drop=True)
    
    # add final match column
    if train:
        df['match'] = df['point_of_interest_1'] == df['point_of_interest_2']
        df.drop(columns=['point_of_interest_1', 'point_of_interest_2'], inplace=True)
    
    return df

In [3]:
train = pd.read_csv(r'C:\Users\caseyrya\Dropbox\foursquare_location_matching_data\train.csv')

In [4]:
train.shape

(1138812, 13)

In [5]:
test_pairs = generate_pairs(train, 5, train = True)

In [ ]:
print(test_pairs.shape)
test_pairs.head(20)

In [ ]:
# positive sample portion
df[df['match']==True].shape[0] / df.shape[0]